## Тест для сравнения двух векторизованных представлений предложений

In [ ]:
import time
from sentence_transformers import SentenceTransformer, util


In [ ]:
class TextComparator:
    def __init__(self, model_name: str = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"):
        """
        Конструктор, загружает модель один раз и хранит её в self.model.
        """
        start_time = time.time()
        self.model = SentenceTransformer(model_name)
        self.load_time = time.time() - start_time  # Время, затраченное на загрузку модели


    def compare_texts(self, text1: str, text2: str) -> dict:
        """
        Сравнивает два текста (text1, text2), возвращая детальную статистику:
        - сходство (cosine similarity),
        - время, затраченное на кодировку каждого текста,
        - время на вычисление сходства,
        - итоговое затраченное время,
        - размеры эмбеддингов.
        """

        total_start = time.time()

        # --- Кодировка text1 ---
        encode1_start = time.time()
        embedding1 = self.model.encode(text1, convert_to_tensor=True)
        encode1_end = time.time()

        # --- Кодировка text2 ---
        encode2_start = time.time()
        embedding2 = self.model.encode(text2, convert_to_tensor=True)
        encode2_end = time.time()

        # --- Вычисление сходства ---
        similarity_start = time.time()
        similarity = util.pytorch_cos_sim(embedding1, embedding2).item()
        similarity_end = time.time()

        total_end = time.time()

        # Формируем детальный отчёт
        report = {
            "text1": text1,
            "text2": text2,
            "similarity": similarity,
            
            "embedding1_shape": tuple(embedding1.shape),
            "embedding2_shape": tuple(embedding2.shape),
            
            "time_encoding_text1": encode1_end - encode1_start,
            "time_encoding_text2": encode2_end - encode2_start,
            "time_similarity_calc": similarity_end - similarity_start,
            
            "total_time_compare": total_end - total_start,
            "model_load_time": self.load_time,  # Время загрузки модели (из конструктора)
        }

        return report

In [ ]:
comparator = TextComparator()


In [ ]:
texts_to_compare = [
    ("Лестница наружная, используемая для эвакуации", "Эвакуационная наружная лестница"),
    ("Кошка сидит на окне", "На окне кошка сидит")
]

for t1, t2 in texts_to_compare:
    result = comparator.compare_texts(t1, t2)
    print("—" * 60)
    print(f"Сравнение:\n  1) {t1}\n  2) {t2}\n")
    for key, value in result.items():
        print(f"{key}: {value}")
    print()